In [62]:
import numpy as np
import pandas as pd
from collections import Counter

class CandleStick:
    def __init__(self, open, high, low, close):
        self.open = open
        self.high = high
        self.low = low
        self.close = close
        self._validate_candle()

    def __repr__(self):
        return f"Candle({self.open}, {self.high}, {self.low}, {self.close})"

    def __str__(self):
        return f"O: {self.open}, H: {self.high}, L: {self.low}, C: {self.close}"

    @property
    def open(self):
        return self._open

    @property
    def high(self):
        return self._high

    @property
    def low(self):
        return self._low

    @property
    def close(self):
        return self._close

    @open.setter
    def open(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError("open must be int or float not {}".format(type(value)))
        if value < 0:
            raise ValueError("open must be positive")
        self._open = value

    @high.setter
    def high(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError("high must be int or float not {}".format(type(value)))
        if value < 0:
            raise ValueError("high must be positive")
        self._high = value

    @low.setter
    def low(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError("low must be int or float not {}".format(type(value)))
        if value < 0:
            raise ValueError("Low must be positive")
        self._low = value

    @close.setter
    def close(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError("close must be int or float not {}".format(type(value)))
        if value < 0:
            raise ValueError("close must be positive")
        self._close = value

    def _validate_candle(self):
        if self.open > self.high:
            raise ValueError("open cannot be greater than high")
        if self.open < self.low:
            raise ValueError("open cannot be less than low")
        if self.close > self.high:
            raise ValueError("close cannot be greater than high")
        if self.close < self.low:
            raise ValueError("close cannot be less than low")

    def type(self):
        if self.open < self.close:
            return 'bullish'
        elif self.open > self.close:
            return 'bearish'
        else:
            return 'doji'

    def candle_stick_size(self):
        return abs(self.high - self.low)

    def body_size(self):
        return abs(self.close - self.open)

    def is_bullish(self):
        return self.type() == 'bullish'

    def is_bearish(self):
        return self.type() == 'bearish'

    def is_doji(self):
        return self.type() == 'doji'

    def candle_stick_body_ratio(self):
        return self.body_size() / self.candle_stick_size()

    # define a method that determines numaricly, where the body is placed in the candlestick # anderer Ansatz suchen!!
    def body_position(self):
        if self.open < self.close:
            return (self.open - self.low) / self.candle_stick_size()
        else:
            return (self.close - self.low) / self.candle_stick_size()










class CandleStickFrame:
    def __init__(self, open, high, low, close):
        open, high, low, close = self._validate_input(open, high, low, close)
        self.candle_sticks = [CandleStick(o, h, l, c) for o, h, l, c in zip(open, high, low, close)]
        self.df = pd.DataFrame({"open": open, "high": high, "low": low, "close": close})
        self._bullish_count, self._bearish_count, self._doji_count = self._type_count()
        self._average_candle_stick_size = self.average_candle_stick_size()
        self._average_body_size = self.average_body_size()

    def __repr__(self):
        return f"CandleFrame({self.df})"

    def __str__(self):
        return f"{self.df}"

    def __len__(self):
        return len(self.candle_sticks)

    def __getitem__(self, index):
        return self.candle_sticks[index]

    def __iter__(self):
        return iter(self.candle_sticks)

    def __reversed__(self):
        return reversed(self.candle_sticks)

    @staticmethod
    def _validate_input(open, high, low, close):
        if not isinstance(open, list) and not isinstance(open, np.ndarray) and not isinstance(open, pd.core.series.Series):
            raise TypeError("open must be list, np.ndarry, or pd.core.series.Series not {}".format(type(open)))
        if not isinstance(high, list) and not isinstance(high, np.ndarray) and not isinstance(high, pd.core.series.Series):
            raise TypeError("high must be list, np.ndarry, or pd.core.series.Series not {}".format(type(high)))
        if not isinstance(low, list) and not isinstance(low, np.ndarray) and not isinstance(low, pd.core.series.Series):
            raise TypeError("low must be list, np.ndarry, or pd.core.series.Series not {}".format(type(low)))
        if not isinstance(close, list) and not isinstance(close, np.ndarray) and not isinstance(close, pd.core.series.Series):
            raise TypeError("close must be list, np.ndarry, or pd.core.series.Series not {}".format(type(close)))
        open = list(open)
        high = list(high)
        low = list(low)
        close = list(close)
        if len(open) != len(high) or len(open) != len(low) or len(open) != len(close):
            raise ValueError("open, high, low, and close must be the same length")
        if not all(isinstance(x, (int, float)) for x in open):
            raise TypeError("open must be list of int or float not {}".format(type(open)))
        if not all(isinstance(x, (int, float)) for x in high):
            raise TypeError("high must be list of int or float not {}".format(type(high)))
        if not all(isinstance(x, (int, float)) for x in low):
            raise TypeError("low must be list of int or float not {}".format(type(low)))
        if not all(isinstance(x, (int, float)) for x in close):
            raise TypeError("close must be list of int or float not {}".format(type(close)))
        return open, high, low, close

    def _type_count(self):
        bullish = 0
        bearish = 0
        doji = 0
        for candle in self.candle_sticks:
            if candle.type() == 'bullish':
                bullish += 1
            elif candle.type() == 'bearish':
                bearish += 1
            elif candle.type() == 'doji':
                doji += 1
        return bullish, bearish, doji

    def _bullish_ratio(self):
        return self._bullish_count / len(self.candle_sticks)

    def _bearish_ratio(self):
        return self._bearish_count / len(self.candle_sticks)

    def _doji_ratio(self):
        return self._doji_count / len(self.candle_sticks)

    def average_candle_stick_size(self):
        return sum([candle.candle_stick_size() for candle in self.candle_sticks]) / len(self.candle_sticks)

    def average_body_size(self):
        return sum([candle.body_size() for candle in self.candle_sticks]) / len(self.candle_sticks)

    def type_ratio(self):
        return 'bullish: {:.2f}, bearish: {:.2f}, doji: {:.2f}'.format(self._bullish_ratio(), self._bearish_ratio(), self._doji_ratio())


class CandleStickPattern:
    def __init__(self, candle_stick_frame):
        self.candle_stick_frame = candle_stick_frame


In [63]:
# load data
df = pd.read_csv('EURUSD_H1.csv')
df

,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,Adj Close,<TICKVOL>,<VOL>,<SPREAD>
0,2015.03.09,14:00:00,1.08603,1.08681,1.08460,1.08578,4992,4.480300e+09,10
1,2015.03.09,15:00:00,1.08584,1.08731,1.08545,1.08582,5974,1.009885e+10,11
2,2015.03.09,16:00:00,1.08582,1.08640,1.08452,1.08454,6175,8.667100e+09,9
3,2015.03.09,17:00:00,1.08454,1.08590,1.08406,1.08456,6638,8.587250e+09,0
4,2015.03.09,18:00:00,1.08455,1.08543,1.08376,1.08465,4574,5.572700e+09,7
...,...,...,...,...,...,...,...,...,...
45774,2022.07.19,18:00:00,1.02450,1.02543,1.02305,1.02382,28840,8.652000e+09,8
45775,2022.07.19,19:00:00,1.02382,1.02445,1.02293,1.02398,20792,6.237600e+09,8
45776,2022.07.19,20:00:00,1.02398,1.02435,1.02340,1.02349,17324,5.197200e+09,8
45777,2022.07.19,21:00:00,1.02349,1.02361,1.02251,1.02314,19953,5.985900e+09,8


In [64]:
# create candlestick frame
candle_stick_frame = CandleStickFrame(df['<OPEN>'], df['<HIGH>'], df['<LOW>'], df['Adj Close'])

In [65]:
for candle in candle_stick_frame:
    if candle.is_doji():
        print(candle)

O: 1.0629, H: 1.0637, L: 1.06259, C: 1.0629
O: 1.11064, H: 1.11147, L: 1.11014, C: 1.11064
O: 1.11072, H: 1.11102, L: 1.11032, C: 1.11072
O: 1.09258, H: 1.0927, L: 1.09152, C: 1.09258
O: 1.11506, H: 1.11527, L: 1.11462, C: 1.11506
O: 1.12641, H: 1.1267, L: 1.12514, C: 1.12641
O: 1.11851, H: 1.11871, L: 1.11808, C: 1.11851
O: 1.10318, H: 1.10358, L: 1.10267, C: 1.10318
O: 1.08369, H: 1.08402, L: 1.08278, C: 1.08369
O: 1.09728, H: 1.09741, L: 1.09647, C: 1.09728
O: 1.10788, H: 1.10842, L: 1.10727, C: 1.10788
O: 1.09815, H: 1.09908, L: 1.09748, C: 1.09815
O: 1.09761, H: 1.09811, L: 1.09722, C: 1.09761
O: 1.10367, H: 1.10484, L: 1.1023, C: 1.10367
O: 1.15225, H: 1.154, L: 1.15078, C: 1.15225
O: 1.12241, H: 1.12261, L: 1.12201, C: 1.12241
O: 1.12814, H: 1.12844, L: 1.12803, C: 1.12814
O: 1.12507, H: 1.12553, L: 1.12363, C: 1.12507
O: 1.12547, H: 1.12575, L: 1.12474, C: 1.12547
O: 1.13761, H: 1.13774, L: 1.13608, C: 1.13761
O: 1.1338, H: 1.1344, L: 1.1335, C: 1.1338
O: 1.09239, H: 1.09281, L

In [61]:
candle_stick_frame.type_ratio()

ValueError: too many values to unpack (expected 2)